In [7]:
import pandas as pd

In [10]:
import sys
sys.path.append('C:\\Users\\deghe\\Documents\\Ironhack\\Week_5\\Mid_bootcamp_project\\mid_project\\src')

In [11]:
from functions import *

In [ ]:
# We read the raw data into a dataframe. We use the separator ";" in order to be able to properly separate the columns
marketing_df=pd.read_csv("C:/Users/e.sansebastian/OneDrive - EXPONDO/Ironhack/Weeks/Week_5/Mid_bootcamp_project/mid_project/data/raw/marketing_campaign_dataset.csv")

In [ ]:
marketing_df.head()

In [ ]:
# We check the shape of the dataframe
print(f"The marketing dataframe contains {marketing_df.shape[0]} rows and {marketing_df.shape[1]} columns.")

In [ ]:
marketing_df.info()

In [ ]:
# Checking at the data we can see some incorrect data, taken into account the content of specific columns.We comment them below:
# Acquisition cost should be float type
# ROI: should be float type
# Date: should be datetime format
# Duration could then considered as a numeric value, but we will leave it as an attribute for now to be able to plot and get that level of granularity

# We will proceed below to clean the data types with functions, when possible or reasonable.

In [ ]:
# First thing we will do is to clean the headers and convert them into snake_case, which will ease then later the manipulation. We will use a function
# called lowercase_headers_and_replace_spaces() from our functions module

In [ ]:
marketing_df=lowercase_headers_and_replace_spaces(marketing_df)
display(marketing_df.columns)

In [ ]:
# Being something punctual we will convert it with the str.replace method, rather than passing it into a function this time, as it may not be very useful in the future.
marketing_df['acquisition_cost'] = marketing_df['acquisition_cost'].str.replace('$', '').str.replace(',', '.').str.replace('.00', '')
marketing_df.head()

In [ ]:
# In order to speed up future data conversions, we pass a function to be able to convert a list of columns to numeric
def columns_to_numeric(df:pd.DataFrame,cols:list)->pd.DataFrame:
    ''' 
    This columns takes a list of columns from a DataFrame and converts them into numeric.
    Input:
    
    df: dataframe we want to clean
    cols: list of columns we want to convert to numeric.

    Output:
    colums properly formatted to numeric.
    '''
    
    df2 = df.copy()

    # we interate over the list of columns we want to convert into numeric
    for col in cols:
        df2[col] = pd.to_numeric(df2[col], errors='coerce')

    return df2

    

In [ ]:
marketing_df=columns_to_numeric(marketing_df,["acquisition_cost","roi"])

In [ ]:
print(marketing_df.dtypes)

In [ ]:
# We next convert the date into date format. I thought about doing it with the next function of extracting mont, quarter, etc., but we should avoid performing multiple actions into 1 column to make it more generic

In [ ]:
marketing_df["date"]=pd.to_datetime(marketing_df['date'])

In [ ]:
# We will create a function to extract the day as day of the week, month, quarter and week of the year. This should help us analyse the seasonality

def date_extract(df:pd.DataFrame,col:"string")->pd.DataFrame:
    '''
    This function takes a DataFrame and extracts the month, quarter and week of the year into new columns

    Input:

    df: The dataframe we want to apply the changes on
    col: The date column we want to extract the month, quarter, week of the year into new colums from

    Output:

    it returns the dataframe with the new 3 columns
    '''
    # We create a copy of our df and extract the corresponding values from the date colum inserted as an input:
    df2=df.copy()
    df2["month"] = df2[col].dt.month
    df2["quarter"] =  df2[col].dt.quarter
    df2["week_of_year"]= df2[col].dt.isocalendar().week

    return df2



In [ ]:
marketing_df = date_extract(marketing_df,"date")

In [ ]:
marketing_df.head()

In [ ]:
display(marketing_df.dtypes)

In [ ]:
display(marketing_df["target_audience"].value_counts())

In [ ]:
# We will split the "demographic" target_audience into 2 columns, provided a specific pattern is present. In this case we will look for digit-digit

import pandas as pd

def extract_pattern(df: pd.DataFrame, col: str, pattern: str, new_col: str) -> pd.DataFrame:
    '''
    This function extracts a pattern from a specified column into a new column.

    Input:
    - df: The DataFrame we want to apply the changes to.
    - col: The column containing the text to search for the pattern.
    - pattern: The regular expression pattern to extract.
    - new_col: The name of the new column to store the extracted patterns.

    Output:
    - Returns the DataFrame with the new column.
    '''
    df2 = df.copy()

    # We use the methode str. to extract the pattern into a new column
    df2[new_col] = df2[col].str.extract(pattern, expand=False)

    # If there are missing values in the original column, fill them in the new column
    df2[new_col].fillna("all ages", inplace=True)

    return df2


In [ ]:
marketing_df=extract_pattern(marketing_df,"target_audience", r'(\d{2}-\d{2})',"age_range")

In [ ]:
# We check the values of the new age-ranges column
marketing_df["age_range"].value_counts()

In [ ]:
marketing_df["target_audience"].value_counts()

In [ ]:
# We now will define a function to clean the remaining target_audience column to only have men, women and all genders

def gender_cleaning(df:pd.DataFrame,col:"string")->pd.DataFrame:
    '''
    This function takes a dataframe as input and returns 3 values for gender, men, women and all genders

    Input: 
    df: dataframe to be modified
    col: name of the column we want to apply the changes on

    Output:

    Returns the dataframe with the 3 options listed above: men, women and all genders.

    '''
    df2=df.copy()
    # Using the lambda function from the data transformation notebook, we adapt the function to the current purpose
    df2 = df.copy()
    df2[col] = df2[col].apply(lambda val: "women" if "women" in str(val).lower() else ("men" if "men" in str(val).lower() else "all genders"))

    return df2

In [ ]:
marketing_df= gender_cleaning(marketing_df,"target_audience")

In [ ]:
marketing_df["target_audience"].value_counts()

In [ ]:
marketing_df.head()

In [ ]:
# We check the columns and data types again
marketing_df.info()

In [ ]:
# We consider the data as clean and save the "cleaned version of the csv into the corresponding folder"

marketing_df.to_csv('C:/Users/e.sansebastian/OneDrive - EXPONDO/Ironhack/Weeks/Week_5/Mid_bootcamp_project/mid_project/data/cleaned/cleaned_marketing_df.csv',index=False)